In [16]:
import socket
import time
import numpy  as np
import matplotlib.pyplot as plt

plt.rc("text", usetex=True)

%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb

In [17]:
class loopy_belief_propagation():
    def __init__(self, pgm):
        if type(pgm) is not factor_graph:
            raise Exception('PGM is not a factor graph')
        if not pgm.is_connected():
            raise Exception('PGM is not connected')
#         if len(pgm.get_graph().es) - 1 == len(pgm.get_graph().vs):
#             raise Exception('PGM is a tree')
        
        self.__t       = 0
        self.__msg     = {}
        self.__msg_new = {}
        self.__pgm     = pgm
        
        # Initialization of messages
        for edge in self.__pgm.get_graph().es:
            start_index, end_index = edge.tuple[0], edge.tuple[1]
            start_name, end_name = self.__pgm.get_graph().vs[start_index]['name'], self.__pgm.get_graph().vs[end_index]['name']
            
            if self.__pgm.get_graph().vs[start_index]['is_factor']:
                self.__msg[(start_name, end_name)] = factor([end_name],   np.array([1.]*self.__pgm.get_graph().vs[end_index]['rank']))
            else:
                self.__msg[(start_name, end_name)] = factor([start_name], np.array([1.]*self.__pgm.get_graph().vs[start_index]['rank']))
            self.__msg[(end_name, start_name)] = self.__msg[(start_name, end_name)]
            
            self.__msg_new[(start_name, end_name)] = 0
            self.__msg_new[(end_name, start_name)] = 0
    
    def belief(self, v_name, num_iter):
        if self.__t > num_iter:
            raise Exception('Invalid number of iterations. Current number: ' + str(self.__t))
        elif self.__t < num_iter:
            self.__loop(num_iter)
#         print("-"*20)
        incoming_messages = []
        for f_name_neighbor in self.__pgm.get_graph().vs[self.__pgm.get_graph().neighbors(v_name)]['name']:
            print("incoming negbious", f_name_neighbor)
            print("message", self.get_factor2variable_msg(f_name_neighbor, v_name).get_distribution())
            incoming_messages.append(self.get_factor2variable_msg(f_name_neighbor, v_name))
        
#         for msg in incoming_messages:
#             print("incomging msg", msg.get_distribution())
        
#         print("join", joint_distribution(incoming_messages).get_distribution())
#         print("nomalized", self.__normalize_msg(joint_distribution(incoming_messages)).get_distribution())
        return self.__normalize_msg(joint_distribution(incoming_messages))
    
    # ----------------------- Variable to factor ------------
    def get_variable2factor_msg(self, v_name, f_name):
        return self.__msg[(v_name, f_name)]
    
    def __compute_variable2factor_msg(self, v_name, f_name):
        incoming_messages = []
#         print("------")
#         print(v_name, f_name)
#         print("incoming")
        for f_name_neighbor in self.__pgm.get_graph().vs[self.__pgm.get_graph().neighbors(v_name)]['name']:
            if f_name_neighbor != f_name:
#                 print("from", f_name_neighbor)
#                 print(self.get_factor2variable_msg(f_name_neighbor, v_name).get_distribution())
                incoming_messages.append(self.get_factor2variable_msg(f_name_neighbor, v_name))
        
        if not incoming_messages:
            return factor([v_name], np.array([1]*self.__pgm.get_graph().vs.find(name=v_name)['rank']))
        else:

#             print("joint", joint_distribution(incoming_messages).get_distribution())
#             print("norm", self.__normalize_msg(joint_distribution(incoming_messages)).get_distribution())
            return self.__normalize_msg(joint_distribution(incoming_messages))
    
    # ----------------------- Factor to variable ------------
    def get_factor2variable_msg(self, f_name, v_name):
        return self.__msg[(f_name, v_name)]
    
    def __compute_factor2variable_msg(self, f_name, v_name):
#         print("------")
#         print(f_name, v_name)
#         print("incoming")
        incoming_messages = [self.__pgm.get_graph().vs.find(f_name)['factor_']]
#         print("self", incoming_messages[0].get_distribution())
        marginalization_variables = []
        for v_name_neighbor in self.__pgm.get_graph().vs[self.__pgm.get_graph().neighbors(f_name)]['name']:
            if v_name_neighbor != v_name:
#                 print("from", v_name_neighbor)
#                 print(self.get_variable2factor_msg(v_name_neighbor, f_name).get_distribution())
                incoming_messages.append(self.get_variable2factor_msg(v_name_neighbor, f_name))
                marginalization_variables.append(v_name_neighbor)

#         print("joint", joint_distribution(incoming_messages).get_distribution())
#         print("marginal", factor_marginalization(
#             joint_distribution(incoming_messages),
#             marginalization_variables
#         ).get_distribution())
#         print("norm", self.__normalize_msg(factor_marginalization(
#             joint_distribution(incoming_messages),
#             marginalization_variables
#         )).get_distribution())
        return self.__normalize_msg(factor_marginalization(
            joint_distribution(incoming_messages),
            marginalization_variables
        ))
    
    # ----------------------- Other -------------------------
    def __loop(self, num_iter):
        # Message updating
        while self.__t < num_iter:
            print("------------------------------------------------------------------------- Iteration", self.__t)
            for edge in self.__pgm.get_graph().es:
                start_index, end_index = edge.tuple[0], edge.tuple[1]
                start_name, end_name   = self.__pgm.get_graph().vs[start_index]['name'], self.__pgm.get_graph().vs[end_index]['name']
                
                
                if self.__pgm.get_graph().vs[start_index]['is_factor']:
                    self.__msg_new[(start_name, end_name)] = self.__compute_factor2variable_msg(start_name, end_name) if start_name != 's' else factor(['s'], np.array([0.5, 0.5]))
                    self.__msg_new[(end_name, start_name)] = self.__compute_variable2factor_msg(end_name, start_name)
                else:
                    self.__msg_new[(start_name, end_name)] = self.__compute_variable2factor_msg(start_name, end_name) if start_name != 's' else factor(['s'], np.array([0.5, 0.5]))
                    self.__msg_new[(end_name, start_name)] = self.__compute_factor2variable_msg(end_name, start_name)
                
                print(start_name, "to", end_name, "message:", self.__msg_new[(start_name, end_name)].get_distribution())
                print(end_name, "to", start_name, "message:", self.__msg_new[(end_name, start_name)].get_distribution())
            self.__msg.update(self.__msg_new)
            self.__t += 1
    
    def __normalize_msg(self, message):
        return factor(message.get_variables(), message.get_distribution()/np.sum(message.get_distribution()))

In [18]:
class belief_propagation():
    def __init__(self, pgm):
        if type(pgm) is not factor_graph:
            raise Exception('PGM is not a factor graph')
        if not (pgm.is_connected() and not pgm.is_loop()):
            raise Exception('PGM is not a tree')
        
        self.__msg = {}
        self.__pgm = pgm
    
    def belief(self, v_name):

        incoming_messages = []
        for f_name_neighbor in self.__pgm.get_graph().vs[self.__pgm.get_graph().neighbors(v_name)]['name']:
            print(f_name_neighbor)
            incoming_messages.append(self.get_factor2variable_msg(f_name_neighbor, v_name))
        
        for incoming_message in incoming_messages:
            print(incoming_message.get_distribution())
            
        return self.__normalize_msg(joint_distribution(incoming_messages))
    
    # ----------------------- Variable to factor ------------
    def get_variable2factor_msg(self, v_name, f_name):
        key = (v_name, f_name)
        if key not in self.__msg:
            self.__msg[key] = factor([v_name], np.array([0.5, 0.5]))
            self.__msg[key] = self.__compute_variable2factor_msg(v_name, f_name)
        return self.__msg[key]
    
    def __compute_variable2factor_msg(self, v_name, f_name):
        incoming_messages = []
        for f_name_neighbor in self.__pgm.get_graph().vs[self.__pgm.get_graph().neighbors(v_name)]['name']:
            if f_name_neighbor != f_name:
                incoming_messages.append(self.get_factor2variable_msg(f_name_neighbor, v_name))

        if not incoming_messages:
            # if the variable does not have its own distribution
            return factor([v_name], np.array([1.]*self.__pgm.get_graph().vs.find(name=v_name)['rank']))
        else:
            # Since all messages have the same dimension (1, order of v_name) the expression after
            # ```return``` is equivalent to ```factor(v_name, np.prod(incoming_messages))```
            return self.__normalize_msg(joint_distribution(incoming_messages))
    
    # ----------------------- Factor to variable ------------
    def get_factor2variable_msg(self, f_name, v_name):
        key = (f_name, v_name)
        if key not in self.__msg:
            self.__msg[key] = self.__compute_factor2variable_msg(f_name, v_name)
        return self.__msg[key]
    
    def __compute_factor2variable_msg(self, f_name, v_name):
        incoming_messages = [self.__pgm.get_graph().vs.find(f_name)['factor_']]

        marginalization_variables = []
        for v_name_neighbor in self.__pgm.get_graph().vs[self.__pgm.get_graph().neighbors(f_name)]['name']:
            if v_name_neighbor != v_name:
                incoming_messages.append(self.get_variable2factor_msg(v_name_neighbor, f_name))
                marginalization_variables.append(v_name_neighbor)
        return self.__normalize_msg(factor_marginalization(
            joint_distribution(incoming_messages),
            marginalization_variables
        ))
    
    # ----------------------- Other -------------------------
    def __normalize_msg(self, message):
        return factor(message.get_variables(), message.get_distribution()/np.sum(message.get_distribution()))

In [19]:
mrf = string2factor_graph("f1(x_1,x_2,x_3)f2(x_1,x_2,x_3)f3(x_2)f4(x_3)")

In [ ]:
prob = [0.95] * 8
prob[6] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2))

for i in range(0, 8):
    indexes = [int(x) for x in list('{0:03b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]][indexes[2]])
    
f1 = factor(['x_1', 'x_2', 'x_3'], prob.copy())

prob = [0.95] * 8
prob[3] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2))

print("-" * 30)
for i in range(0, 8):
    indexes = [int(x) for x in list('{0:03b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]][indexes[2]])
    
f2 = factor(['x_1', 'x_2', 'x_3'], prob.copy())

prob = [0.95] * 2
prob[0] = 0.05
prob[1] = 0.95

prob = np.array(prob)
prob = prob.reshape((2))

print("-" * 30)
for i in range(0, 2):
    indexes = [int(x) for x in list('{0:01b}'.format(i))]
    print(indexes, prob[indexes[0]])
    
f3 = factor(['x_2'], prob.copy())

prob = [0.95] * 2
prob[1] = 0.05
prob[0] = 0.95
prob = np.array(prob)
prob = prob.reshape((2))

print("-" * 30)
for i in range(0, 2):
    indexes = [int(x) for x in list('{0:01b}'.format(i))]
    print(indexes, prob[indexes[0]])
    
f4 = factor(['x_3'], prob.copy())

# f4 = factor(['r1'], np.array([1-r1, r1]))
# f5 = factor(['r2'], np.array([1-r2, r2]))
# f6 = factor(['w'], np.array([1-w, w]))
# f7 = factor(['p'], np.array([1-p, p]))

In [ ]:
mrf.change_factor_distribution('f1', f1)
mrf.change_factor_distribution('f2', f2)
mrf.change_factor_distribution('f3', f3)
mrf.change_factor_distribution('f4', f4)

In [ ]:
lbp = loopy_belief_propagation(mrf)
tol = []

# for i in range(15):
#     tol.append(np.linalg.norm(lbp.belief('b', i).get_distribution() - exact))
print("x_1", lbp.belief('x_1', 20).get_distribution())

In [ ]:
exact = factor_marginalization(joint_distribution([f1, f2, f3, f4]), ['x_2', 'x_3']).get_distribution()
exact = exact / np.sum(exact)
exact

In [20]:
mrf = string2factor_graph("f1(x_1,s)f2(x_1,s)f3(x_2,s)f4(x_2,s)f5(x_1)f6(x_2)")

In [27]:
prob = [0.95] * 4
prob[2] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2))

for i in range(0, 4):
    indexes = [int(x) for x in list('{0:02b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]])
    
f1 = factor(['x_1', 's'], prob.copy())

prob = [0.95] * 4
prob[1] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2))

print("-" * 30)
for i in range(0, 4):
    indexes = [int(x) for x in list('{0:02b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]])
    
f2 = factor(['x_1', 's'], prob.copy())


prob = [0.95] * 4
prob[2] = 0.05

prob = np.array(prob)
prob = prob.reshape((2, 2))

print("-" * 30)
for i in range(0, 4):
    indexes = [int(x) for x in list('{0:02b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]])
    
f3 = factor(['x_2', 's'], prob.copy())

prob = [0.95] * 4
prob[1] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2))

print("-" * 30)
for i in range(0, 4):
    indexes = [int(x) for x in list('{0:02b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]])

f4 = factor(['x_2', 's'], prob.copy())
r1 = 0.95
r2 = 0.2
w = 0.23
f5 = factor(['x_1'], np.array([1-r1, r1]))
f6 = factor(['x_2'], np.array([1-r2, r2]))

# f4 = factor(['r1'], np.array([1-r1, r1]))
# f5 = factor(['r2'], np.array([1-r2, r2]))
# f6 = factor(['w'], np.array([1-w, w]))
# f7 = factor(['p'], np.array([1-p, p]))

[0, 0] 0.95
[0, 1] 0.95
[1, 0] 0.05
[1, 1] 0.95
------------------------------
[0, 0] 0.95
[0, 1] 0.05
[1, 0] 0.95
[1, 1] 0.95
------------------------------
[0, 0] 0.95
[0, 1] 0.95
[1, 0] 0.05
[1, 1] 0.95
------------------------------
[0, 0] 0.95
[0, 1] 0.05
[1, 0] 0.95
[1, 1] 0.95


In [28]:
mrf.change_factor_distribution('f1', f1)
mrf.change_factor_distribution('f2', f2)
mrf.change_factor_distribution('f3', f3)
mrf.change_factor_distribution('f4', f4)
mrf.change_factor_distribution('f5', f5)
mrf.change_factor_distribution('f6', f6)

In [29]:
lbp = loopy_belief_propagation(mrf)
tol = []

# for i in range(15):
#     tol.append(np.linalg.norm(lbp.belief('b', i).get_distribution() - exact))
print("s", lbp.belief('s', 20).get_distribution())

------------------------------------------------------------------------- Iteration 0
x_1 to f1 message: [0.5 0.5]
f1 to x_1 message: [0.65517241 0.34482759]
s to f1 message: [0.5 0.5]
f1 to s message: [0.34482759 0.65517241]
x_1 to f2 message: [0.5 0.5]
f2 to x_1 message: [0.34482759 0.65517241]
s to f2 message: [0.5 0.5]
f2 to s message: [0.65517241 0.34482759]
x_2 to f3 message: [0.5 0.5]
f3 to x_2 message: [0.65517241 0.34482759]
s to f3 message: [0.5 0.5]
f3 to s message: [0.34482759 0.65517241]
x_2 to f4 message: [0.5 0.5]
f4 to x_2 message: [0.34482759 0.65517241]
s to f4 message: [0.5 0.5]
f4 to s message: [0.65517241 0.34482759]
x_1 to f5 message: [0.5 0.5]
f5 to x_1 message: [0.05 0.95]
x_2 to f6 message: [0.5 0.5]
f6 to x_2 message: [0.8 0.2]
------------------------------------------------------------------------- Iteration 1
x_1 to f1 message: [0.02695418 0.97304582]
f1 to x_1 message: [0.65517241 0.34482759]
s to f1 message: [0.5 0.5]
f1 to s message: [0.34482759 0.655172

In [8]:
bp = belief_propagation(mrf)

In [9]:
print("s", bp.belief('s').get_distribution())

f1
f2
f3
f4
[0.18395383 0.81604617]
[0.74581527 0.25418473]
[0.48516223 0.51483777]
[0.94018848 0.05981152]
s [0.90738738 0.09261262]


In [10]:
exact = factor_marginalization(joint_distribution([f1, f2, f3, f4, f5, f6]), ['x_1', 'x_2']).get_distribution()
exact = exact / np.sum(exact)
exact

array([0.78168495, 0.21831505])

In [ ]:
mrf = string2factor_graph("f1(a,b,c,d)f2(a,b,c,e)f3(a,b,c,d,e)f4(a,b,c,d,e)f5(a,b,c,d,e)f6(b)f7(c)f8(d)f9(e)")

In [ ]:
prob = [0.95] * 16
prob[14] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2, 2))

for i in range(0, 16):
    indexes = [int(x) for x in list('{0:04b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]][indexes[2]][indexes[3]])
    
f1 = factor(['a', 'b', 'c', 'd'], prob.copy())

prob = [0.95] * 16
prob[14] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2, 2))

for i in range(0, 16):
    indexes = [int(x) for x in list('{0:04b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]][indexes[2]][indexes[3]])
    
f2 = factor(['a', 'b', 'c', 'e'], prob.copy())

prob = [0.95] * 32
prob[15] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2, 2, 2))

for i in range(0, 32):
    indexes = [int(x) for x in list('{0:05b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]][indexes[2]][indexes[3]][indexes[4]])
    
f3 = factor(['a', 'b', 'c', 'd', 'e'], prob.copy())

prob = [0.95] * 32
prob[23] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2, 2, 2))

for i in range(0, 32):
    indexes = [int(x) for x in list('{0:05b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]][indexes[2]][indexes[3]][indexes[4]])
    
f4 = factor(['a', 'b', 'c', 'd', 'e'], prob.copy())

prob = [0.95] * 32
prob[27] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2, 2, 2))

for i in range(0, 32):
    indexes = [int(x) for x in list('{0:05b}'.format(i))]
    print(indexes, prob[indexes[0]][indexes[1]][indexes[2]][indexes[3]][indexes[4]])
    
f5 = factor(['a', 'b', 'c', 'd', 'e'], prob.copy())

r1 = 1
r2 = 1
r3 = 1
r4 = 1

f6 = factor(['b'], np.array([1-r1, r1]))
f7 = factor(['c'], np.array([1-r1, r1]))
f8 = factor(['d'], np.array([1-r1, r1]))
f9 = factor(['e'], np.array([1-r1, r1]))
# prob = [0.95] * 4
# prob[1] = 0.05
# prob = np.array(prob)
# prob = prob.reshape((2, 2))

# print("-" * 30)
# for i in range(0, 4):
#     indexes = [int(x) for x in list('{0:02b}'.format(i))]
#     print(indexes, prob[indexes[0]][indexes[1]])
    
# f3 = factor(['b', 's'], prob.copy())

# prob = [0.95] * 4
# prob[2] = 0.05
# prob = np.array(prob)
# prob = prob.reshape((2, 2))

# print("-" * 30)
# for i in range(0, 4):
#     indexes = [int(x) for x in list('{0:02b}'.format(i))]
#     print(indexes, prob[indexes[0]][indexes[1]])
    
# f4 = factor(['b', 's'], prob.copy())

# prob = [0.95] * 4
# prob[1] = 0.05
# prob = np.array(prob)
# prob = prob.reshape((2, 2))

# print("-" * 30)
# for i in range(0, 4):
#     indexes = [int(x) for x in list('{0:02b}'.format(i))]
#     print(indexes, prob[indexes[0]][indexes[1]])
    
# f5 = factor(['c', 's'], prob.copy())

# prob = [0.95] * 4
# prob[2] = 0.05
# prob = np.array(prob)
# prob = prob.reshape((2, 2))

# print("-" * 30)
# for i in range(0, 4):
#     indexes = [int(x) for x in list('{0:02b}'.format(i))]
#     print(indexes, prob[indexes[0]][indexes[1]])
    
# f6 = factor(['c', 's'], prob.copy())

# r1 = 0.75
# r2 = 0.01
# r3 = 0.6

# f7 = factor(['a'], np.array([1-r1, r1]))
# f8 = factor(['b'], np.array([1-r2, r2]))
# f9 = factor(['c'], np.array([1-r3, r3]))



In [ ]:
mrf.change_factor_distribution('f1', f1)
mrf.change_factor_distribution('f2', f2)
mrf.change_factor_distribution('f3', f3)
mrf.change_factor_distribution('f4', f4)
mrf.change_factor_distribution('f5', f5)
mrf.change_factor_distribution('f6', f6)
mrf.change_factor_distribution('f7', f7)
mrf.change_factor_distribution('f8', f8)
mrf.change_factor_distribution('f9', f9)

In [ ]:
lbp = loopy_belief_propagation(mrf)
tol = []

# for i in range(15):
#     tol.append(np.linalg.norm(lbp.belief('b', i).get_distribution() - exact))
print("a", lbp.belief('a', 20).get_distribution())

In [ ]:
exact = factor_marginalization(joint_distribution([f1, f2, f3]), ['a']).get_distribution()
exact = exact / np.sum(exact)
exact

In [11]:
l = np.array(['a'], dtype)

In [15]:
str(l[0]).startswith('a')

True